In [ ]:
%%capture
!pip install tensorboardX simpletransformers sklearn


In [ ]:

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging
import sklearn
import pandas as pd

df = pd.read_csv("datasets/problem_statements.csv")

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

df2 = df.sample(frac=1)
# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
size = len(df2)
train_cutoff = round(size*0.8)
train_df= df2[:train_cutoff]
eval_df = df2[train_cutoff:]

model_args = ClassificationArgs(save_steps=-1, save_model_every_epoch=False, sliding_window=True)

model = ClassificationModel('roberta', 'roberta-base',args=model_args, use_cuda=False) # You can set class weights by using the optional weight argument


In [ ]:
# Train the model
model.train_model(train_df, overwrite_output_dir=True)

# Evaluate the model, u
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score )